In [18]:
import os
import pandas as pd
import pathlib
import glob
import pickle
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
# import libraries
import re
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

random_seed = 10
warnings.filterwarnings('ignore')

# set working dFectory
working_dFectory = r"/home/dr/phia data"
os.chdF(working_dFectory)

# load data
phia_male = pickle.load(open("Transformed_data\phia_male_step1.pkl", 'rb'))
phia_female = pickle.load(open("Transformed_data\phia_female_step1.pkl", 'rb'))
pd.set_option("display.max.columns", None)
pd.set_option("display.max.rows", None)
phia_male = pd.DataFrame(phia_male)
phia_female = pd.DataFrame(phia_female)
M = phia_male
F = phia_female



In [19]:
M_C_D = pd.get_dummies(M, columns=['country'], prefix='Country')
F_C_D = pd.get_dummies(F, columns=['country'], prefix='Country')

# Rename country
country_list = ['Tanzania', 'Malawi', 'Swaziland', 'Zambia']
i = 0
for colonne in list(M_C_D.columns):
    if re.match('Country', colonne):
        M_C_D.rename(columns={colonne: country_list[i]}, inplace=True)
        i += 1

country_list = ['Tanzania', 'Malawi', 'Swaziland', 'Zambia']
i = 0
for colonne in list(F_C_D.columns):
    if re.match('Country', colonne):
        F_C_D.rename(columns={colonne: country_list[i]}, inplace=True)
        i += 1        

M_C_D = M_C_D.astype(str).astype(float)
F_C_D = F_C_D.astype(str).astype(float)


In [ ]:
 # remove variables with very low variance 

constant_filter = VarianceThreshold(threshold=0.01)
constant_filter.fit(F_C_D)
len(F_C_D.columns[constant_filter.get_support()])
constant_columns = [column for column in F_C_D.columns if column not in F_C_D.columns[constant_filter.get_support()]]
print(constant_columns)
F_C_D.drop(labels=constant_columns, axis=1, inplace=True)


constant_filter = VarianceThreshold(threshold=0.01)
constant_filter.fit(M_C_D)
len(M_C_D.columns[constant_filter.get_support()])
constant_columns = [column for column in M_C_D.columns if column not in M_C_D.columns[constant_filter.get_support()]]
print(constant_columns)
M_C_D.drop(labels=constant_columns, axis=1, inplace=True)


In [ ]:
F_C_D.drop(columns='HIV status final', inplace=True)
M_C_D.drop(columns='HIV status final', inplace=True)


# M_correlation_matrix  
M_correlated_features = set()
F_correlated_features = set()
M_correlation_matrix = M_C_D.corr()
F_correlation_matrix = F_C_D.corr()
# # remove columns for correlation above 0.8
for i in range(len(M_correlation_matrix.columns)):
    for j in range(i):
        if abs(M_correlation_matrix.iloc[i, j]) > 0.8:
            colname =M_correlation_matrix.columns[i]
            M_correlated_features.add(colname)
len(M_correlated_features)

for i in range(len(F_correlation_matrix.columns)):
    for j in range(i):
        if abs(F_correlation_matrix.iloc[i, j]) > 0.8:
            colname =F_correlation_matrix.columns[i]
            F_correlated_features.add(colname)
# len(F_correlated_features)


In [21]:
M_C_D.drop(['90 with ARV data',
 'ART Recode for 90-90-90 with ARV data',
 'Aware for 90-90-90 with ARV data',
 'Awareness combining self-report',
 'Duration of time on ART',
 'Ever tested HIV',
 'LAg: recent or long term infection ARV results',
 'Marital status',
 'Self-Reported HIV Status of HIV',
 'Tested HIV and receive results',
 'Union type',
 'Used condom at last sex in past 12 months',
 'VFal load suppressed',
 'Wealth index',
 'Whether ARVs detected',
 'age group',
 'all past 12 months sex partners spouse or live-in',
 'last sex partner spouse or live-in in last 12 months'],axis=1, inplace=True)
F_C_D.drop(['90 with ARV data',
 'ART Recode for 90-90-90 with ARV data',
 'Age group',
 'All past 12 months sex partners spouse or live-in',
 'Aware for 90-90-90 with ARV data',
 'Awareness combining self-report',
 'Current marital status',
 'Delivered 3 years preceeding survey',
 'Duration of time on ART',
 'Ever tested HIV',
 'LAg: recent or long term infection ARV results',
 'Last sex partner spouse or live-in in last 12 months',
 'Pregnancy status',
 'Self-Reported HIV Status of HIV',
 'Self-reported HIV status during last pregnancy',
 'Tested HIV and receive results',
 'Union type',
 'Used condom at last sex in past 12 months',
 'VFal load suppressed',
 'Wealth index','respondent is a mother'],axis=1, inplace=True)



In [ ]:
from tableone import TableOne
import pandas as pd
columns =['Age', 'Relationship with family head', 'Respondent live in household',
       'Sick to work last three months', 'Ever attended school',
       'Ever enrolled in school', 'Highest level of education',
       'Highest grade at that school level',
       'Work for payment in last 12 months', 'Ever married or lived together',
       'Avoiding pregnancy', 'CFcumcision status', 'Age at fFst sex',
       'Ever sought TB treatment', 'Alcohol drink frequency',
       'Urban area indicator', 'Known HIV status', 'Wealth quintile',
       'HIV status final', 'Sexual intercourse in the past 12 months',
       'Ever had sexual intercourse', 'Bought or sold sex in the past 12 months',
       'Condom was used at last paid sex in past 12 months',
       'last sex partner relations past 12 months',
       'LAg: recent or long term infection', 'CD4 category', 'on ART',
       'ART initiated past 12 months those ever on ART', 'Tanzania', 'Malawi',
       'Swaziland', 'Zambia']
categorical =['Relationship with family head', 'Respondent live in household',
       'Sick to work last three months', 'Ever attended school',
       'Ever enrolled in school', 'Highest level of education',
       'Highest grade at that school level',
       'Work for payment in last 12 months', 'Ever married or lived together',
       'Avoiding pregnancy', 'CFcumcision status', 
       'Ever sought TB treatment', 'Alcohol drink frequency',
       'Urban area indicator', 'Known HIV status', 'Wealth quintile',
       'Sexual intercourse in the past 12 months',
       'Ever had sexual intercourse', 'Bought or sold sex in the past 12 months',
       'Condom was used at last paid sex in past 12 months',
       'last sex partner relations past 12 months',
       'LAg: recent or long term infection', 'CD4 category', 'on ART',
       'ART initiated past 12 months those ever on ART', 'Tanzania', 'Malawi',
       'Swaziland', 'Zambia']
        
# nonnormal = [''Age','Age at fFst sex',]

groupby = 'HIV status final'
maletable = TableOne(M_C_D,columns= columns,categorical=categorical,
                   groupby =groupby,pval= False)

print(maletable.tabulate(tablefmt="xlsx"))

In [ ]:
M_C_D_col_names = M_C_D.columns.drop('HIV status final')
F_C_D_col_names = F_C_D.columns.drop('HIV status final')
CM_train = {}
CM_Y_train = {}
CM_test = {}
CM_Y_test = {}

for name in list(country_list):
    CM_train[name] = M_C_D[M_C_D[name]!=1].astype('float')
    CM_test[name] = M_C_D[M_C_D[name]==1].astype('float')
    CM_Y_train[name] = M_C_D['HIV status final'][M_C_D[name]!=1].astype('float')
    CM_Y_test[name] = M_C_D['HIV status final'][M_C_D[name]==1].astype('float')
    CM_train[name].drop(columns='HIV status final', inplace=True)
    CM_test[name].drop(columns='HIV status final', inplace=True)

CF_train = {}
CF_test = {}
CF_Y_train = {}
CF_Y_test = {}

for name in list(country_list):
    CF_train[name] = F_C_D[F_C_D[name]!=1].astype('float')
    CF_test[name] = F_C_D[F_C_D[name]==1].astype('float')
    CF_Y_train[name] = F_C_D['HIV status final'][F_C_D[name]!=1].astype('float')
    CF_Y_test[name] = F_C_D['HIV status final'][F_C_D[name]==1].astype('float')
    CF_train[name].drop(columns='HIV status final', inplace=True)
    CF_test[name].drop(columns='HIV status final', inplace=True)
    
M_X_train_entFe = {}
M_X_test_entFe = {}
M_Y_train_entFe = {}
M_Y_test_entFe = {}

F_X_train_entFe = {}
F_X_test_entFe = {}
F_Y_train_entFe = {}
F_Y_test_entFe = {}


# split between train (80%) and test (20%) with stratification
for name in list(country_list):  
    M_X_train_entFe[name], M_X_test_entFe[name], M_Y_train_entFe[name],
    M_Y_test[name] = train_test_split(CM_train[name], CM_Y_train[name], test_size=0.2, stratify=CM_Y_train[name],random_state=random_seed)
    
# split between train (80%) and test (20%) with stratification
for name in list(country_list):  
    F_X_train_entFe[name], F_X_test_entFe[name], F_Y_train_entFe[name],F_Y_test_entFe[name] = train_test_split(CF_train[name], CF_Y_train[name], test_size=0.2, stratify=CF_Y_train[name],random_state=random_seed)
    
# standardization of the data
M_scaler = StandardScaler(with_mean=False)
M_X_train_entFe = round(pd.DataFrame(M_scaler.fit_transform(M_X_train_entFe), columns=M_C_D_col_names), 2)
M_X_test_entFe = round(pd.DataFrame(M_scaler.transform(M_X_test_entFe), columns=M_C_D_col_names), 2)
M_var_entFe = M_scaler.var_

# F_scaler = StandardScaler(with_mean=False)
F_X_train_entFe = round(pd.DataFrame(F_scaler.fit_transform(F_X_train_entFe), columns=F_C_D_col_names), 2)
F_X_test_entFe = round(pd.DataFrame(F_scaler.transform(F_X_test_entFe), columns=F_C_D_col_names), 2)
F_var_entFe = F_scaler.var_

f = open("pure_data\M_X_train_entFe.pkl", 'wb')
pickle.dump(M_X_train_entFe, f)
f = open("pure_data\M_Y_train_entFe.pkl", 'wb')
pickle.dump(M_Y_train_entFe, f)
f = open("pure_data\M_X_test_entFe.pkl", 'wb')
pickle.dump(M_X_test_entFe, f)
f = open("pure_data\M_Y_test_entFe.pkl", 'wb')
pickle.dump(M_Y_test_entFe, f)

f = open("pure_data\F_X_train_entFe.pkl", 'wb')
pickle.dump(F_X_train_entFe, f)
f = open("pure_data\F_Y_train_entFe.pkl", 'wb')
pickle.dump(F_Y_train_entFe, f)

f = open("pure_data\F_X_test_entFe.pkl", 'wb')
pickle.dump(F_X_test_entFe, f)
f = open("pure_data\F_Y_test_entFe.pkl", 'wb')
pickle.dump(F_Y_test_entFe, f)

f.close()

    

In [56]:
#================================================================================================================================
# subsampling 3 countries out of 4
#================================================================================================================================

CM_train = {}
CM_test = {}
CM_Y_train = {}
CM_Y_test = {}

for name in list(country_list):
    CM_train[name] = M_C_D[M_C_D[name]!=1].astype('float')
    CM_test[name] = M_C_D[M_C_D[name]==1].astype('float')
    CM_Y_train[name] = M_C_D['HIV status final'][M_C_D[name]!=1].astype('float')
    CM_Y_test[name] = M_C_D['HIV status final'][M_C_D[name]==1].astype('float')
    CM_train[name].drop(columns='HIV status final', inplace=True)
    CM_test[name].drop(columns='HIV status final', inplace=True)
    
    
#================================================================================================================================
# subsampling 3 countries out of 4
#================================================================================================================================

CF_train = {}
CF_test = {}
CF_Y_train = {}
CF_Y_test = {}

for name in list(country_list):
    CF_train[name] = F_C_D[F_C_D[name]!=1].astype('float')
    CF_test[name] = F_C_D[F_C_D[name]==1].astype('float')
    CF_Y_train[name] = F_C_D['HIV status final'][F_C_D[name]!=1].astype('float')
    CF_Y_test[name] = F_C_D['HIV status final'][F_C_D[name]==1].astype('float')
    CF_train[name].drop(columns='HIV status final', inplace=True)
    CF_test[name].drop(columns='HIV status final', inplace=True)

In [ ]:
   
#================================================================================================================================
# impute and standardize
#================================================================================================================================

M_X_train = {}
M_X_test = {}
M_Y_train = {}
M_Y_test = {}

F_X_train = {}
F_X_test = {}
F_Y_train = {}
F_Y_test = {}


# split between train (80%) and test (20%) with stratification
for name in list(country_list):  
    M_X_train[name], M_X_test[name], M_Y_train[name], M_Y_test[name] = train_test_split(CM_train[name], CM_Y_train[name], test_size=0.2, stratify=CM_Y_train[name], random_state=random_seed)
    
# split between train (80%) and test (20%) with stratification
for name in list(country_list):  
    F_X_train[name], F_X_test[name], F_Y_train[name], F_Y_test[name] = train_test_split(CF_train[name], CF_Y_train[name], test_size=0.2, stratify=CF_Y_train[name], random_state=random_seed)
    
# multiple imputations using chained equations
n_imputations = 5
M_X_train_comp = {}
M_X_test_comp = {}
CM_test_comp = {}
M_X_train_imp = {}
M_X_test_imp = {}
CM_test_imp = {}


n_imputations = 5
F_X_train_comp = {}
F_X_test_comp = {}
CF_test_comp = {}
F_X_train_imp = {}
F_X_test_imp = {}
CF_test_imp = {}

for name in list(country_list):
    M_X_train_comp[name] = []
    M_X_test_comp[name] = []
    CM_test_comp[name] = []
    
for name in list(country_list):
    F_X_train_comp[name] = []
    F_X_test_comp[name] = []
    CF_test_comp[name] = []
    
    for i in range(n_imputations):
        print(f'Current imputed country is {name}')
        print(f'Current imputation round is {i+1}/{n_imputations}')
        M_imputer = IterativeImputer(sample_posterior=True, random_state=i, verbose=1)
        M_X_train_comp[name].append(M_imputer.fit_transform(M_X_train[name]))
        M_X_test_comp[name].append(M_imputer.transform(M_X_test[name]))
        CM_test_comp[name].append(M_imputer.transform(CM_test[name]))
        
    for i in range(n_imputations):
        print(f'Current imputed country is {name}')
        print(f'Current imputation round is {i+1}/{n_imputations}')
        F_imputer = IterativeImputer(sample_posterior=True, random_state=i, verbose=1)
        F_X_train_comp[name].append(F_imputer.fit_transform(F_X_train[name]))
        F_X_test_comp[name].append(F_imputer.transform(F_X_test[name]))
        CF_test_comp[name].append(F_imputer.transform(CF_test[name]))
        
    M_X_train_imp[name] = pd.DataFrame(np.mean(M_X_train_comp[name], axis=0), columns=M_C_D_col_names)
    M_X_test_imp[name] = pd.DataFrame(np.mean(M_X_test_comp[name], axis=0), columns=M_C_D_col_names)
    CM_test_imp[name] = pd.DataFrame(np.mean(CM_test_comp[name], axis=0), columns=M_C_D_col_names)
    
    
    F_X_train_imp[name] = pd.DataFrame(np.mean(F_X_train_comp[name], axis=0), columns=F_C_D_col_names)
    F_X_test_imp[name] = pd.DataFrame(np.mean(F_X_test_comp[name], axis=0), columns=F_C_D_col_names)
    CF_test_imp[name] = pd.DataFrame(np.mean(CF_test_comp[name], axis=0), columns=F_C_D_col_names)
    
# imputation processing
def min_max_int(df, column, nb_cat=float('inf')):
    df[column] = round(df[column], 0)
    df[column][df[column] < 0] = 0
    df[column][df[column] > nb_cat - 1] = nb_cat - 1
    return df

def onehot(df, name):
    onehot = pd.DataFrame()
    for colonne in list(df.columns):
        if re.match(name, colonne):
            for indice in df[colonne][~df[colonne].isin([0, 1])].index:
                onehot.loc[indice, colonne] = df.loc[indice, colonne]
    i = 0
    for indice in onehot.index:
        onehot.loc[indice, list(onehot.idxmax(axis=1))[i]] = 1
        i += 1
    onehot[onehot!=1] = 0
    df.update(onehot)
    return df

M_X_train_proc =  {}
M_X_test_proc =  {}
CM_test_proc =  {}

F_X_train_proc =  {}
F_X_test_proc =  {}
CF_test_proc =  {}
        
        
for name in list(country_list):
    
    def M_imp_process(df):
    
        min_max_int(df, 'Age')
        onehot(df, 'Relationship with family head')
        min_max_int(df, 'Respondent live in household', 2)
        min_max_int(df, 'Known HIV status', 2)
        min_max_int(df, 'Bought/sold sex in the past 12 months', 2)
        min_max_int(df, 'Condom was used at last paid sex in past 12 months', 2)
        min_max_int(df, 'LAg: recent/long term infection', 2)
        min_max_int(df, 'CD4 category', 5)
        min_max_int(df, 'on ART', 2)
        min_max_int(df, 'ART initiated past 12 months those ever on ART', 2)
        onehot(df, 'last sex partner relations past 12 months')
        min_max_int(df, 'Sick to work last three months', 3)
        min_max_int(df, 'Ever attended school',2)    
        min_max_int(df, 'Ever enrolled in school', 2)
        onehot(df, 'Highest level of education')    
        min_max_int(df, 'Highest grade at that school level')
        min_max_int(df, 'Work for payment in last 12 months',2)
        min_max_int(df, 'Ever married/lived together',2)
        min_max_int(df, 'Avoiding pregnancy',3)    
        min_max_int(df, 'CFcumcision status',2)
        min_max_int(df, 'Age at fFst sex')      
        min_max_int(df, 'Ever sought TB treatment', 2)    
        onehot(df, 'Alcohol drink frequency')    
        min_max_int(df, 'Urban area indicator',2)      
        onehot(df, 'Wealth quintile')    
        min_max_int(df, 'Sexual intercourse in the past 12 months', 2)    
        min_max_int(df, 'Ever had sexual intercourse', 2) 
        
        return df
    
for name in list(country_list):
    M_X_train_proc[name] = M_imp_process(M_X_train_imp[name])
    M_X_test_proc[name] = M_imp_process(M_X_test_imp[name])
    CM_test_proc[name] = M_imp_process(CM_test_imp[name])
  
                  
def F_imp_process(df):
    min_max_int(df, 'Age')
    onehot(df, 'Current marital status')
    min_max_int(df, 'Number of pregnancies')
    min_max_int(df, 'Self-reported ARV status during pregnancy',2)
    min_max_int(df, 'Bought/sold sex in the past 12 months',2)
    min_max_int(df, 'Condom was used at last paid sex in past 12 months',2)
    onehot(df, 'last sex partner relations past 12 months')
    min_max_int(df, 'CD4 category',2)
    min_max_int(df, 'On ART',2)
    min_max_int(df, 'ART initiated past 12 months those ever on ART',2)
    min_max_int(df, 'Whether ARVs detected',2)
    min_max_int(df, 'LAg: recent/long term infection',2)
    min_max_int(df, 'Breastfed last child',2)
    min_max_int(df, 'Delivered 12 months preceeding survey',2)
    min_max_int(df, 'Ever had a pregnancy that resulted in a live bFth',2)
    min_max_int(df, 'No. of childen given bFth since 2012')
    min_max_int(df, 'Pregnacy status now') 
    min_max_int(df, 'Known HIV status',2)
    onehot(df, 'Relationship with family head')
    min_max_int(df, 'Respondent live in household', 2)
    min_max_int(df, 'Sick to work last three months', 3)
    min_max_int(df, 'Ever attended school',2)    
    min_max_int(df, 'Ever enrolled in school', 2)
    onehot(df, 'Highest level of education')    
    min_max_int(df, 'Highest grade at that school level')
    min_max_int(df, 'Work for payment in last 12 months',2)
    min_max_int(df, 'Ever married/lived together',2)
    min_max_int(df, 'Avoiding pregnancy',3)    
    min_max_int(df, 'Age at fFst sex')      
    min_max_int(df, 'tested HIV at pregnancy',2)
    min_max_int(df, 'Ever sought TB treatment', 2)    
    onehot(df, 'Alcohol drink frequency')    
    min_max_int(df, 'Urban area indicator',2)      
    onehot(df, 'Wealth quintile')    
    min_max_int(df, 'Sexual intercourse past 12 months', 2)    
    min_max_int(df, 'Ever had sexual intercourse', 2)    
     
    return df

for name in list(country_list):
    F_X_train_proc[name] = F_imp_process(F_X_train_imp[name])
    F_X_test_proc[name] = F_imp_process(F_X_test_imp[name])
    CF_test_proc[name] = F_imp_process(CF_test_imp[name])

    
M_X_train_ready = {}
M_X_test_ready = {}
CM_test_ready = {}

F_X_train_ready = {}
F_X_test_ready = {}
CF_test_ready = {}


# standardization of the data
for name in list(country_list):
    M_scaler = StandardScaler(with_mean=False)
    M_X_train_ready[name] = round(pd.DataFrame(M_scaler.fit_transform(M_X_train_proc[name]), columns=M_C_D_col_names), 2)
    M_X_test_ready[name] = round(pd.DataFrame(M_scaler.transform(M_X_test_proc[name]), columns=M_C_D_col_names), 2)
    CM_test_ready[name] = round(pd.DataFrame(M_scaler.transform(CM_test_proc[name]), columns=M_C_D_col_names), 2)
    M_var = M_scaler.var_
   
# standardization of the data
for name in list(country_list):
    F_scaler = StandardScaler(with_mean=False)
    F_X_train_ready[name] = round(pd.DataFrame(F_scaler.fit_transform(F_X_train_proc[name]), columns=F_C_D_col_names), 2)
    F_X_test_ready[name] = round(pd.DataFrame(F_scaler.transform(F_X_test_proc[name]), columns=F_C_D_col_names), 2)
    CF_test_ready[name] = round(pd.DataFrame(F_scaler.transform(CF_test_proc[name]), columns=F_C_D_col_names), 2)
    F_var = F_scaler.var_  
    
# pickling data
f = open("Imputation_data\M_X_train_imp.pkl", 'wb')
pickle.dump(M_X_train_imp, f)

# pickling data
f = open("Imputation_data\F_X_train_imp.pkl", 'wb')
pickle.dump(F_X_train_imp, f)
f = open("Imputation_data\M_X_test_imp.pkl", 'wb')
pickle.dump(M_X_test_imp, f)

f = open("Imputation_data\F_X_test_imp.pkl", 'wb')
pickle.dump(F_X_test_imp, f)
f = open("Imputation_data\CM_test_imp.pkl", 'wb')
pickle.dump(CM_test_imp, f)

f = open("Imputation_data\CF_test_imp.pkl", 'wb')
pickle.dump(CF_test_imp, f)

f = open("Imputation_data\M_X_train_comp.pkl", 'wb')
pickle.dump(M_X_train_comp, f)

f = open("Imputation_data\F_X_train_comp.pkl", 'wb')
pickle.dump(F_X_train_comp, f)

f = open("Imputation_data\M_X_test_comp.pkl", 'wb')
pickle.dump(M_X_test_comp, f)

f = open("Imputation_data\F_X_test_comp.pkl", 'wb')
pickle.dump(F_X_test_comp, f)
f = open("Imputation_data\CM_test_comp.pkl", 'wb')
pickle.dump(CM_test_comp, f)

f = open("Imputation_data\CF_test_comp.pkl", 'wb')
pickle.dump(CF_test_comp, f)

f = open("Imputation_data\M_X_train_proc.pkl", 'wb')
pickle.dump(M_X_train_proc, f)

f = open("Imputation_data\F_X_train_proc.pkl", 'wb')
pickle.dump(F_X_train_proc, f)

f = open("Imputation_data\M_X_test_proc.pkl", 'wb')
pickle.dump(M_X_test_proc, f)

f = open("Imputation_data\F_X_test_proc.pkl", 'wb')
pickle.dump(F_X_test_proc, f)

f = open("Imputation_data\CM_test_proc.pkl", 'wb')
pickle.dump(CM_test_proc, f)


f = open("Imputation_data\CF_test_proc.pkl", 'wb')
pickle.dump(CF_test_proc, f)

f = open("Train_samples\M_X_train_ready.pkl", 'wb')
pickle.dump(M_X_train_ready, f)

f = open("Train_samples\F_X_train_ready.pkl", 'wb')
pickle.dump(F_X_train_ready, f)

f = open("Test_samples\M_X_test_ready.pkl", 'wb')
pickle.dump(M_X_test_ready, f)

f = open("Test_samples\F_X_test_ready.pkl", 'wb')
pickle.dump(F_X_test_ready, f)


f = open("Left_one_out_samples\CM_test_ready.pkl", 'wb')
pickle.dump(CM_test_ready, f)

f = open("Left_one_out_samples\CF_test_ready.pkl", 'wb')
pickle.dump(CF_test_ready, f)

f = open("Train_samples\M_Y_train.pkl", 'wb')
pickle.dump(M_Y_train, f)

f = open("Train_samples\F_Y_train.pkl", 'wb')
pickle.dump(F_Y_train, f)


f = open("Test_samples\M_Y_test.pkl", 'wb')
pickle.dump(M_Y_test, f)

f = open("Test_samples\F_Y_test.pkl", 'wb')
pickle.dump(F_Y_test, f)


f = open("Left_one_out_samples\CM_Y_test.pkl", 'wb')
pickle.dump(CM_Y_test, f)\


f = open("Left_one_out_samples\CF_Y_test.pkl", 'wb')
pickle.dump(CF_Y_test, f)

f = open("Imputation_data\M_var.pkl", 'wb')
pickle.dump(M_var, f)

f = open("Imputation_data\F_var.pkl", 'wb')
pickle.dump(F_var, f)

f.close()